# EXPLORATION NOTEBOOK

## Cell 1 — Title & Overview

### AutoCode Clinic – Exploration Notebook  
Understanding the Multi-Agent Architecture

This notebook provides a detailed breakdown of the internal design of **AutoCode Clinic**, an automated debugging and patching system built using a **multi-agent workflow**.

It explains:

- The role of each agent  
- System architecture  
- Workflow loop  
- How patches are generated  
- How validation and sandbox execution work  

Use this notebook to understand how all components fit together before using the system or contributing to the project.

## Cell 2 — Architecture Diagram (Markdown Only)

### System Architecture Diagram

Below is the high-level architecture of the AutoCode Clinic:

In [ ]:
    ┌──────────────────────┐
    │     Start Session    │
    └───────────┬──────────┘
                v
    ┌────────────────────────┐
    │ Static Analysis Agent  │
    │ - scan code structure  │
    └───────────┬────────────┘
                v
    ┌────────────────────────┐
    │ Validation Agent       │
    │ - execute tests        │
    └───────┬────────────────┘
            │
  Tests OK? │
     YES ───┴──► Status: FIXED
            │
            NO
            v
    ┌────────────────────────┐
    │ Patch Generation Agent │
    │ - generate JSON patch  │
    └───────┬────────────────┘
            v
    ┌────────────────────────┐
    │ Apply Patch            │
    └───────┬────────────────┘
            v
    ┌────────────────────────┐
    │ Orchestrator Iteration │
    └───────────┬────────────┘
                │
                └──► Loop until fixed or max_iters

**Key idea:**  
The system runs in a **loop** — validate → patch → validate — until tests pass or iterations run out.

## Cell 3 — Load Orchestrator (Code)

In [ ]:
from agents.orchestrator import OrchestratorAgent

orchestrator = OrchestratorAgent()
orchestrator

## Cell 4 — Agent Descriptions

### Agent Responsibilities

#### 🔹 StaticAnalysisAgent
- Performs lightweight AST-based static checks  
- Detects syntax errors  
- Flags unsafe constructs like `eval()`  

#### 🔹 ValidationAgent
- Executes code safely inside an isolated sandbox  
- Captures stdout, stderr, and exceptions  
- Determines whether tests pass  

#### 🔹 PatchGenerationAgent
- Constructs a prompt with:
  - test output  
  - static analysis results  
  - file contents  
- Uses the `mock_model()` to simulate patch generation  
- Returns a JSON patch object  

#### 🔹 OrchestratorAgent
- Manages the entire workflow  
- Performs multiple iterations  
- Applies patches  
- Logs artifacts into session state  

## Cell 5 — Workflow Visualization (Markdown Only)

### Multi-Agent Workflow Loop

In [ ]:
 +----------------------+
 |   Start Debugging    |
 +----------+-----------+
            |
            v
 +---------------------------+
 | StaticAnalysisAgent       |
 | • Scan code               |
 | • Detect structural issues|
 +------------+--------------+
              |
              v
 +---------------------------+
 | ValidationAgent           |
 | • Execute tests           |
 | • Capture errors          |
 +------------+--------------+
              |
          Tests pass? 
          |        |
          v        v
         YES      NO
          |        |
          v        |
        FIXED      |
                   v
      +---------------------------+
      | PatchGenerationAgent      |
      | • Model-based patching    |
      | • Produces JSON patch     |
      +------------+--------------+
                  |
                  v
      +---------------------------+
      | Apply Patch to Code       |
      +------------+--------------+
                  |
                  v
      +---------------------------+
      | Orchestrator (Loop Back)  |
      +------------+--------------+
                  |
                  v
     (Continue until fixed or max_iters)

## Cell 6 — Example Job Setup (Code)

In [ ]:
import uuid

code_files = {"solution.py": "def add(a, b):\n    return a - b\n"}
tests = """
import solution

def test_add():
    assert solution.add(2,3) == 5
"""

session_id = "explore-" + str(uuid.uuid4())

## Cell 7 — Run One Iteration Manually (Code)

In [ ]:
orchestrator.handle_job(session_id, code_files.copy(), tests, max_iters=1)

## Cell 8 — Display Session History (Code)

In [ ]:
from src.utils.session_state import state_manager
import json

history = state_manager.get(session_id)
print(json.dumps(history, indent=2)[:1500])